In [118]:
def get_reads(file_name):
    with open (file_name) as file:
        count = 0
        reads = []
        for line in file:
            if count == 0: 
                count += 1
                continue
            if count == 1:
                reads.append(line.strip())
                count +=1
                continue
            if count == 2:
                count += 1
                continue
            if count == 3:
                count = 0
        return reads       
                
def extract_kmers(reads, k):
    kmers = []
    for i in range(0,len(reads)-k+1):
        kmers.append(reads[i:i+k])
        
    return kmers    

def update_graph_out(out,kmers,k):
    for i in range (0,len(kmers)-1):
        if kmers[i] not in out:
             out[kmers[i]] = {}
        if kmers[i+1] not in out[kmers[i]]:
            out[kmers[i]][kmers[i+1]] = [0,kmers[i]+kmers[i+1][k-1:]]
        out[kmers[i]][kmers[i+1]][0] += 1
    return out  

def update_graph_incom(incom,kmers,k):
    for i in range (0,len(kmers)-1):
        if kmers[i+1] not in incom:
             incom[kmers[i+1]] = {}
        if kmers[i] not in incom[kmers[i+1]]:
            incom[kmers[i+1]][kmers[i]] = [0,kmers[i]+kmers[i+1][k-1:]]
        incom[kmers[i+1]][kmers[i]][0] += 1
    return incom 
        
def create_de_brujn_graph_from_fastq(file_name,k):
    graph_out = {}
    graph_in = {}
    reads = get_reads(file_name)
    for read in reads:
        kmers = extract_kmers(read,k)
        graph_out = update_graph_out(graph_out,kmers,k)
        graph_in = update_graph_incom(graph_in,kmers,k)
    return (graph_out,graph_in, k)

def graph_compression(graph_out,graph_in,k):
    for current in list (graph_out.keys()):
        new_coverage = 0.0
        left_coverage = ''
        right_coverage = ''
        res_string = ''
        if current not in graph_in:
            continue
        if len(graph_out[current]) == 1 and len(graph_in[current]) == 1:
            previous = list (graph_in[current].keys())[0]
            next_ = list (graph_out[current].keys())[0]
            left_sequence = graph_out[previous][current][1]
            right_sequence = graph_out[current][next_][1]
            left_coverage = graph_out[previous][current][0]
            right_coverage = graph_out[current][next_][0]
            new_sequence = left_sequence + right_sequence[k:]
            new_coverage = (len(left_sequence)*left_coverage + len(right_sequence)*right_coverage)/float(len(new_sequence))
            (graph_out[previous])[next_] = [new_coverage, new_sequence]
            (graph_in[next_])[previous] = [new_coverage, new_sequence]
            del graph_out[previous][current]
            del graph_in[next_][current]       
            del graph_out[current]       
            del graph_in[current]
            
    return (graph_out, graph_in)

def get_av(graph_out):
    av_coverage = 0
    av_len = 0
    total_edge = 0
    for key in graph_out:
        for key1 in graph_out[key]:
            av_coverage += graph_out[key][key1][0]
            av_len += len(graph_out[key][key1][1])
            total_edge += 1
    return (av_coverage / total_edge, av_len / total_edge)
            
def remove_tips(graph_out,graph_in):
    av_coverage, av_len = get_av(graph_out)
    flag = False
    
    graph_in_keys = list(graph_in.keys())
    
    for key in graph_in_keys:
        if key in graph_out or len(graph_in[key]) > 1:
            continue
            
        parent_key = list(graph_in[key].keys())[0]
        edge = graph_in[key][parent_key]
            
        if edge[0] < av_coverage / 3 and len(edge[1]) < av_len / 3:
            del graph_out[parent_key][key]
            del graph_in[key]
            flag = True
                
    return flag
            
                
def dump_graph(outgoing, viz_fname):
    with open(viz_fname, 'w') as out_f:
        print('digraph ag{', file=out_f)
        for left, dict in outgoing.items():
            for right in dict:
                round_coverage = dict[right][0]
                print(left + '[label="{}"]'.format(left), file=out_f)
                print(right + '[label="{}"]'.format(right), file=out_f)
                print(
                    left + ' -> ' + right +
                    '[label="C = {}"]'.format(round_coverage),
                    file=out_f)
        print('}', file=out_f)


                    

graph_out, graph_in, k = create_de_brujn_graph_from_fastq('/Users/vvpetrov/Desktop/Python_Bio/s_6.first1000.fastq', 14)

graph_compression(graph_out,graph_in,k)

while remove_tips(graph_out,graph_in):
    graph_compression(graph_out,graph_in,k)

dump_graph (graph_out,'/Users/vvpetrov/Desktop/Python_Bio/graph.txt')


In [ ]:
def dump_graph(outgoing, viz_fname):
    with open(viz_fname, 'w') as out_f:
        print('digraph ag{', file=out_f)
        for left, dict in outgoing.items():
            for right in dict:
                round_coverage = dict[right]
                print(left + '[label="{}"]'.format(left), file=out_f)
                print(right + '[label="{}"]'.format(right), file=out_f)
                print(
                    left + ' -> ' + right +
                    '[label="C = {}"]'.format(round_coverage),
                    file=out_f)
        print('}', file=out_f)



In [97]:
def get_avg(graph_out):
    total_coverage = 0
    total_len = 0
    total_edge = 0
    for key in graph_out:
        graph_out1 = graph_out[key]
        for key1 in graph_out1:
            key1 = graph_out1[key1]
            total_coverage += key1[0]
            total_len += len(key1)
            total_edge += 1
    av_coverage = total_coverage/total_edge
    av_len = total_len/total_edge
    
    return (av_coverage, av_len)
                  
    
    
            

(4.333333333333333, 2.0)